In [1]:
import pandas as pd
import numpy as np
import random

def clean_separate_df(df, yr):
    all_players = df[df.year==yr][['player','pos','adp','ppr_pts','year']].sort_values('adp').reset_index(drop=True)
    qb = all_players[all_players.pos=='QB']
    rb = all_players[all_players.pos=='RB']
    wr = all_players[all_players.pos=='WR']
    te = all_players[all_players.pos=='TE']
    return all_players, qb, rb, wr, te


def create_strats(strat):
    if strat == 'h':
        return ['hero_rb', ['RB','TE','QB','WR','WR','WR','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'z':
        return ['zero_rb', ['WR','WR','TE','QB','VAL','VAL','RB','RB','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'r':
        return ['rb_heavy', ['RB','RB','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    else:
        return ['val',['VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    
def create_owners_strats(hero,zero,rb_heavy,value):
    strats = []
    for i in range(0,hero):
        strats.append(create_strats('hero'))
    for i in range(0,zero):
        strats.append(create_strats('zero'))
    for i in range(0,rb_heavy):
        strats.append(create_strats('rb_heavy'))
    for i in range(0,value):
        strats.append(create_strats('value'))
    return strats

def one_random_strat():
    '''h - hero, z - zero, r - rb_heavy, v - value'''
    return random.choice(['h','z','r','v','v','v'])

def create_league():
    league_draft_strat= []
    for i in range(0,12):
        league_draft_strat.append(one_random_strat())
        
    strats = []
    for l in league_draft_strat:
        strats.append(create_strats(l))
        
    return strats

class Cheatsheet:
    def __init__(self, all_players, qb, rb, wr, te):
        self.all_players = all_players
        self.qb = qb
        self.rb = rb
        self.wr = wr
        self.te = te
      
    def draft_qb(self,owner):
        selection = self.qb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.qb = self.qb[1:]
        owner.qb_count+=1
        return selection
        
    def draft_rb(self,owner):
        selection = self.rb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.rb = self.rb[1:]
        owner.rb_count+=1
        return selection
    
    def draft_wr(self,owner):
        selection = self.wr.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.wr = self.wr[1:]
        owner.wr_count+=1
        return selection
    
    def draft_te(self,owner):
        selection = self.te.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.te = self.te[1:]
        owner.te_count+=1
        return selection
    
    def draft_not_qb(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'QB':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_te(owner)
    
    def draft_not_te(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'TE':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_qb(owner)
        
    def draft_not_wr(self,owner):
        pass
    
    def draft_value(self,owner): 
        selection = self.all_players.iloc[0,:]
        if self.all_players.iloc[0,:].pos == 'QB':
            self.qb = self.qb.drop(index = selection.name)
            owner.qb_count+=1
        elif self.all_players.iloc[0,:].pos == 'RB':
            self.rb = self.rb.drop(index = selection.name)
            owner.rb_count+=1
        elif self.all_players.iloc[0,:].pos == 'WR':
            self.wr = self.wr.drop(index = selection.name)
            owner.wr_count+=1
        else:
            self.te = self.te.drop(index = selection.name)
            owner.te_count+=1
        self.all_players = self.all_players.iloc[1:]
        return selection
    

class Team_owner:
    def __init__(self, strat, draft_pos, draft_num):
        self.team = []
        self.strat_name = strat[0]      
        self.strat_order = strat[1]
        self.draft_pos = draft_pos
        self.draft_num = draft_num
        self.qb_count = 0
        self.te_count = 0
        self.wr_count = 0
        self.rb_count = 0
        
        
    def draft_player(self,cheatsheet):
        if len(self.team) == 13 and self.te_count == 0: #Draft a TE in 10th round if no TEs
            self.strat_order.pop(0)
            return cheatsheet.draft_te(self)
        
        elif len(self.team) == 8 and self.qb_count == 0: #Draft a QB in 9th round if no QBs
            self.strat_order.pop(0)
            return cheatsheet.draft_qb(self)
        
        elif len(self.team) == 7 and self.rb_count == 1: #Draft a RB in 8th round if only 1 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 1: #Draft a WR in 8th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif len(self.team) == 6 and self.rb_count == 0: #Draft a RB in 7th round if only 0 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 0: #Draft a WR in 7th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif self.qb_count == 2 and self.strat_order[0]=='VAL': #Don't draft more than 2 QB
            self.strat_order.pop(0)
            return cheatsheet.draft_not_qb(self)
        
        elif self.te_count == 2 and self.strat_order[0]=='VAL': # Don't draft more than 2 TE
            self.strat_order.pop(0)
            return cheatsheet.draft_not_te(self)
        
        
#         elif self.rb_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_wr(self)
        
#         elif self.wr_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_rb(self)
        
        else:
            pos = self.strat_order.pop(0)
            if pos == 'QB':
                self.qb_count+=1
                return cheatsheet.draft_qb(self)
            elif pos == 'RB':
                return cheatsheet.draft_rb(self)
            elif pos == 'WR':
                return cheatsheet.draft_wr(self)
            elif pos == 'TE':
                return cheatsheet.draft_te(self)
            else:
                return cheatsheet.draft_value(self)



            
    

In [2]:
def top7(df):
    '''
    Takes in a team owner's team and returns their best starters for the season:
    1 QB, 2 RB, 2 WR, 1 RB or WR, 1 TE for a total of 7 players 
    '''
    top = []
    for season in range(2010,2023):
        for sim in range(0,100):
            for i in range(1,13):
                top7 = [] 
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='QB')].sort_values('ppr_pts',ascending=False).head(1))
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2))
                df = df.drop(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2).index)
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2))
                df = df.drop(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2).index)
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & ((df.pos=='RB')|(df.pos=='WR'))].sort_values('ppr_pts',ascending=False).head(1))
                top7.append(df[(df.year==season)&(df.draft_num==sim)&(df.draft_pos==i) & (df.pos=='TE')].sort_values('ppr_pts',ascending=False).head(1))
                top.append(pd.concat(top7))
    return pd.concat(top)

# Create 100 mock leagues each year to run analysis on

In [3]:
df = pd.read_csv('season1.csv',index_col=0)
owners_teams = [] 
for season in range(2010,2023):
    for sim in range(0,100):
        all_players, qb, rb, wr, te = clean_separate_df(df,season)
        strats = create_league()
        cheat = Cheatsheet(all_players, qb, rb, wr, te)

        owners = [] #Create 12 team owners with 'value_strat' drafting strategy
        for i, s in zip(range(0,12),strats.copy()):
            owners.append(Team_owner(s,i+1,sim))

        owners.reverse()
        for i in range(0,14): #Draft 14 rounds for each team owner
            owners.reverse()
            for o in owners:
                o.team.append(o.draft_player(cheat))

        owners_teams.append(owners)

all_teams = []
for t in owners_teams:
    for o in t:
        df = pd.concat(o.team,axis=1).T
        df['draft_pos'] = o.draft_pos
        df['draft_strat'] = o.strat_name
        df['draft_num'] = o.draft_num
        all_teams.append(df) 

df = pd.concat(all_teams)
    
   

In [4]:
def top7(df):
    '''
    Takes in a team owner's team and returns their best starters for the season:
    1 QB, 2 RB, 2 WR, 1 RB or WR, 1 TE for a total of 7 players 
    '''
    top7 = [] 
    top7.append(df[df.pos=='QB'].sort_values('ppr_pts',ascending=False).head(1))
    top7.append(df[df.pos=='RB'].sort_values('ppr_pts',ascending=False).head(2))
    df = df.drop(df[df.pos=='RB'].sort_values('ppr_pts',ascending=False).head(2).index)
    top7.append(df[df.pos=='WR'].sort_values('ppr_pts',ascending=False).head(2))
    df = df.drop(df[df.pos=='WR'].sort_values('ppr_pts',ascending=False).head(2).index)
    top7.append(df[(df.pos=='RB')|(df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(1))
    top7.append(df[df.pos=='TE'].sort_values('ppr_pts',ascending=False).head(1))
    df = pd.concat(top7)
    df['total_pts'] = df.ppr_pts.sum()
    return df

In [5]:
com = []
for seas in range(2010,2023): # for 'year'
    for num in range(0,100): #for 'draft_num'
        for i in range(1,13): #for 'draft_pos'
            temp = df[(df.year==seas)&(df.draft_num==num)&(df.draft_pos==i)]
            com.append(top7(temp))
top = pd.concat(com) #df of each team owners top 7 starters with 'total_pts' column

In [6]:
top.to_csv('top7.csv')
df.to_csv('sim100.csv')

In [ ]:
df[(df.year==2022) & (df.draft_pos==1)& (top.draft_strat=='hero_rb')].head(14)

/var/folders/fz/0282wvb93rn0lm_p0nscw1sm0000gn/T/ipykernel_59577/3093038176.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[(df.year==2022) & (df.draft_pos==1)& (top.draft_strat=='hero_rb')].head(14)


In [ ]:
top[(top.year==2022) & (top.draft_pos==1)& (top.draft_strat=='hero_rb')].head(14)

In [7]:
top.total_pts.mean()

1495.5528435628698

In [8]:
top.groupby('draft_strat')['total_pts'].mean().sort_values()

draft_strat
rb_heavy    1470.991637
val         1498.045357
zero_rb     1503.085400
hero_rb     1505.406606
Name: total_pts, dtype: float64

In [9]:
top.groupby('draft_pos')['total_pts'].mean()

draft_pos
1     1511.209463
2     1519.281538
3     1517.876437
4     1518.104572
5     1503.051769
6     1506.665308
7     1485.668037
8     1494.564139
9     1503.595846
10    1493.691692
11    1444.482385
12    1448.447308
Name: total_pts, dtype: float64

In [10]:
top.groupby(['draft_pos','draft_strat'])[['total_pts']].mean().sort_values(['draft_pos','total_pts'])

total_pts
draft_pos draft_strat             
1         rb_heavy     1478.704027
          val          1496.232532
          zero_rb      1526.489641
          hero_rb      1569.838208
2         rb_heavy     1513.352766
          val          1518.406881
          zero_rb      1521.416000
          hero_rb      1527.265591
3         rb_heavy     1482.637874
          zero_rb      1506.374380
          val          1525.177812
          hero_rb      1539.181057
4         rb_heavy     1498.523526
          zero_rb      1501.223697
          val          1526.343049
          hero_rb      1530.358883
5         hero_rb      1462.020635
          rb_heavy     1504.184071
          zero_rb      1509.510837
          val          1512.124780
6         hero_rb      1482.133641
          rb_heavy     1497.463927
          zero_rb      1514.709211
          val          1515.320126
7         rb_heavy     1466.057729
          val          1485.381245
          zero_rb      1488.147475
          hero_rb      1503.091743
8         rb_heavy     1469.808658
          val          1492.862000
          zero_rb      1500.428846
          hero_rb      1517.079039
9         rb_heavy     1480.536406
          zero_rb      1502.554187
          val          1508.698077
          hero_rb      1512.253922
10        rb_heavy     1437.241892
          hero_rb      1482.976636
          val          1510.423148
          zero_rb      1512.131019
11        rb_heavy     1397.234802
          val          1445.402564
          zero_rb      1458.429817
          hero_rb      1481.329167
12        rb_heavy     1424.065550
          val          1442.691876
          hero_rb      1456.284649
          zero_rb      1487.185484

In [11]:
top.groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

mean  count
draft_pos draft_strat                    
1         hero_rb      1569.838208   1484
3         hero_rb      1539.181057   1589
4         hero_rb      1530.358883   1379
2         hero_rb      1527.265591   1302
1         zero_rb      1526.489641   1757
4         val          1526.343049   4683
3         val          1525.177812   4480
2         zero_rb      1521.416000   1575
          val          1518.406881   4578
8         hero_rb      1517.079039   1603
6         val          1515.320126   4452
          zero_rb      1514.709211   1596
2         rb_heavy     1513.352766   1645
9         hero_rb      1512.253922   1428
10        zero_rb      1512.131019   1512
5         val          1512.124780   4774
10        val          1510.423148   4536
5         zero_rb      1509.510837   1421
9         val          1508.698077   4732
3         zero_rb      1506.374380   1694
5         rb_heavy     1504.184071   1582
7         hero_rb      1503.091743   1526
9         zero_rb      1502.554187   1421
4         zero_rb      1501.223697   1477
8         zero_rb      1500.428846   1456
4         rb_heavy     1498.523526   1560
6         rb_heavy     1497.463927   1533
1         val          1496.232532   4368
8         val          1492.862000   4550
7         zero_rb      1488.147475   1386
12        zero_rb      1487.185484   1302
7         val          1485.381245   4724
10        hero_rb      1482.976636   1498
3         rb_heavy     1482.637874   1336
6         hero_rb      1482.133641   1519
11        hero_rb      1481.329167   1344
9         rb_heavy     1480.536406   1519
1         rb_heavy     1478.704027   1490
8         rb_heavy     1469.808658   1490
7         rb_heavy     1466.057729   1462
5         hero_rb      1462.020635   1323
11        zero_rb      1458.429817   1526
12        hero_rb      1456.284649   1596
11        val          1445.402564   4641
12        val          1442.691876   4739
10        rb_heavy     1437.241892   1554
12        rb_heavy     1424.065550   1463
11        rb_heavy     1397.234802   1589

In [13]:
top[top.draft_strat=='hero_rb'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count','max','min']).\
sort_values('mean',ascending=False)

,,mean,count,max,min
draft_pos,draft_strat,,,,
1,hero_rb,1569.838208,1484,1961.4,1149.3
3,hero_rb,1539.181057,1589,2022.7,1033.2
4,hero_rb,1530.358883,1379,2006.7,1094.2
2,hero_rb,1527.265591,1302,2083.6,935.1
8,hero_rb,1517.079039,1603,2039.8,1030.4
9,hero_rb,1512.253922,1428,2032.6,997.5
7,hero_rb,1503.091743,1526,1985.5,1030.7
10,hero_rb,1482.976636,1498,2033.4,953.7
6,hero_rb,1482.133641,1519,1962.7,1038.3


In [14]:
top[top.draft_strat=='zero_rb'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
1,zero_rb,1526.489641,1757
2,zero_rb,1521.416000,1575
6,zero_rb,1514.709211,1596
10,zero_rb,1512.131019,1512
5,zero_rb,1509.510837,1421
3,zero_rb,1506.374380,1694
9,zero_rb,1502.554187,1421
4,zero_rb,1501.223697,1477
8,zero_rb,1500.428846,1456


In [16]:
top[top.draft_strat=='val'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
4,val,1526.343049,4683
3,val,1525.177812,4480
2,val,1518.406881,4578
6,val,1515.320126,4452
5,val,1512.124780,4774
10,val,1510.423148,4536
9,val,1508.698077,4732
1,val,1496.232532,4368
8,val,1492.862000,4550


In [17]:
top[top.draft_strat=='rb_heavy'].groupby(['draft_pos','draft_strat'])['total_pts'].agg(['mean','count']).sort_values('mean',ascending=False)

,,mean,count
draft_pos,draft_strat,,
2,rb_heavy,1513.352766,1645
5,rb_heavy,1504.184071,1582
4,rb_heavy,1498.523526,1560
6,rb_heavy,1497.463927,1533
3,rb_heavy,1482.637874,1336
9,rb_heavy,1480.536406,1519
1,rb_heavy,1478.704027,1490
8,rb_heavy,1469.808658,1490
7,rb_heavy,1466.057729,1462
